In [1]:
from vilt.utils.write_nlvr2 import make_arrow
import json
from tqdm import tqdm
from collections import defaultdict
import pyarrow as pa
import pandas as pd

In [2]:
root = '.'

In [3]:
train_data = list(
    map(json.loads, open(f"{root}/nlvr2/data/train.json").readlines())
)
test1_data = list(
    map(json.loads, open(f"{root}/nlvr2/data/test1.json").readlines())
)
dev_data = list(map(json.loads, open(f"{root}/nlvr2/data/dev.json").readlines()))

In [4]:
splits = [
    "train",
    "dev",
    "test1",
]
datas = [
    train_data,
    dev_data,
    test1_data,
]

In [5]:
annotations = dict()

for split, data in zip(splits, datas):
    _annot = defaultdict(list)
    for row in tqdm(data):
        _annot["-".join(row["identifier"].split("-")[:-1])].append(row)
    annotations[split] = _annot

100%|█| 86373/86373 [00:00<00:00, 168324.61it/
100%|█| 6967/6967 [00:00<00:00, 406897.01it/s]


In [6]:
annotations['test1']['test1-0-1']

[{'validation': {'28': 'False'},
  'sentence': 'There is an empty glass.',
  'left_url': 'http://www.belgiansmaak.com/wp-content/gallery/belgian-beers-post/dynamic/31.-Tilquin.jpg-nggid041318-ngg0dyn-0x0x100-00f0w010c010r110f110r010t010.jpg',
  'writer': '103',
  'label': 'False',
  'right_url': 'https://www.craftbrewingbusiness.com/wp-content/uploads/2017/09/unnamed-1.jpg',
  'synset': 'beer bottle',
  'query': 'group of beer bottles41',
  'identifier': 'test1-0-1-0',
  'extra_validations': {'56': 'False',
   '83': 'False',
   '19': 'False',
   '92': 'False'}},
 {'validation': {'127': 'False'},
  'sentence': 'The right image shows three bottles of beer lined up.',
  'left_url': 'http://www.belgiansmaak.com/wp-content/gallery/belgian-beers-post/dynamic/31.-Tilquin.jpg-nggid041318-ngg0dyn-0x0x100-00f0w010c010r110f110r010t010.jpg',
  'writer': '80',
  'label': 'False',
  'right_url': 'https://www.craftbrewingbusiness.com/wp-content/uploads/2017/09/unnamed-1.jpg',
  'synset': 'beer bottle

In [7]:
def process(root, iden, row):
    texts = [r["sentence"] for r in row]
    labels = [r["label"] for r in row]

    split = iden.split("-")[0]

    if iden.startswith("train"):
        directory = row[0]["directory"]
        path = f"{root}/images/train/{directory}/{iden}"
    else:
        path = f"{root}/{split}/{iden}"

    with open(f"run.png", "rb") as fp:
        img0 = fp.read()
    with open(f"run.png", "rb") as fp:
        img1 = fp.read()

    return [img0, img1, texts, labels, iden]

In [ ]:
split='train'
bs = [
    process(root, iden, row) for iden, row in tqdm(annotations[split].items())
]

  1%|   | 292/51585 [00:00<00:17, 2915.18it/s]

In [15]:
dataframe = pd.DataFrame(
            bs, columns=["image_0", "image_1", "questions", "answers", "identifier"],
)

In [16]:
dataframe

,image_0,image_1,questions,answers,identifier
0,b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...,b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...,[The right image shows a curving walkway of da...,"[False, False]",dev-850-0
1,b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...,b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...,[The right image shows a curving walkway of da...,"[True, True]",dev-850-2
2,b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...,b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...,[The right image shows a curving walkway of da...,"[False, False]",dev-850-3
3,b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...,b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...,[The right image shows a curving walkway of da...,"[True, True]",dev-850-1
4,b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...,b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...,[There are exactly six bottles in the right im...,"[True, False]",dev-896-0
...,...,...,...,...,...
4046,b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...,b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...,"[In each image, a woman is drying her face wit...",[False],dev-893-3
4047,b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...,b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...,"[In each image, a woman is drying her face wit...",[True],dev-893-0
4048,b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...,b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...,[The left and right image contains the same nu...,"[True, True]",dev-946-1
4049,b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...,b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...,[The left and right image contains the same nu...,"[True, True]",dev-946-0


In [17]:
table = pa.Table.from_pandas(dataframe)

In [18]:
with pa.OSFile(f"dataset/nlvr2_{split}.arrow", "wb") as sink:
    with pa.RecordBatchFileWriter(sink, table.schema) as writer:
        writer.write_table(table)

In [19]:
from vilt.datasets.base_dataset import BaseDataset
import sys
import random

class NLVR2Dataset(BaseDataset):
    def __init__(self, *args, split="", **kwargs):
        assert split in ["train", "val", "test"]
        self.split = split

        if split == "train":
            names = ["nlvr2_train"]
        elif split == "val":
            names = ["nlvr2_dev", "nlvr2_test1"]
        elif split == "test":
            names = ["nlvr2_dev", "nlvr2_test1"]

        super().__init__(
            *args,
            **kwargs,
            names=names,
            text_column_name="questions",
            remove_duplicate=False,
        )

    def __getitem__(self, index):
        result = None
        while result is None:
            try:
                image_tensor_0 = self.get_image(index, image_key="image_0")["image"]
                image_tensor_1 = self.get_image(index, image_key="image_1")["image"]
                text = self.get_text(index)["text"]
                result = True
            except:
                print(
                    f"error while read file idx {index} in {self.names[0]}",
                    file=sys.stderr,
                )
                index = random.randint(0, len(self.index_mapper) - 1)

        index, question_index = self.index_mapper[index]
        answers = self.table["answers"][index][question_index].as_py()
        answers = answers == "True"

        return {
            "image_0": image_tensor_0,
            "image_1": image_tensor_1,
            "text": text,
            "answers": answers,
            "table_name": self.table_names[index],
        }


In [20]:
from vilt.datamodules.datamodule_base import BaseDataModule

class NLVR2DataModule(BaseDataModule):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    @property
    def dataset_cls(self):
        return NLVR2Dataset

    @property
    def dataset_name(self):
        return "nlvr2"

In [21]:
# Get tokenizer from
import torch
from transformers import (
    DataCollatorForLanguageModeling,
    DataCollatorForWholeWordMask,
    BertTokenizer,
)

def get_pretrained_tokenizer(from_pretrained):
    if torch.distributed.is_initialized():
        if torch.distributed.get_rank() == 0:
            BertTokenizer.from_pretrained(
                from_pretrained, do_lower_case="uncased" in from_pretrained
            )
        torch.distributed.barrier()
    return BertTokenizer.from_pretrained(
        from_pretrained, do_lower_case="uncased" in from_pretrained
    )
tokenizer = get_pretrained_tokenizer("bert-base-uncased")

In [22]:
%load_ext autoreload
%autoreload 2
from vilt.datasets.nlvr2_dataset import NLVR2Dataset
n = NLVR2Dataset(data_dir='dataset', split='train', transform_keys=["pixelbert_randaug"],image_size=384)
n.tokenizer = tokenizer

In [23]:
len(n)

6967

In [33]:
n.table

pyarrow.Table
image_0: binary
image_1: binary
questions: list<item: string>
  child 0, item: string
answers: list<item: string>
  child 0, item: string
identifier: string

In [34]:
n[5]

{'image_0': [tensor([[[-1., -1., -1.,  ..., -1., -1., -1.],
           [-1., -1., -1.,  ..., -1., -1., -1.],
           [-1., -1., -1.,  ..., -1., -1., -1.],
           ...,
           [-1., -1., -1.,  ..., -1., -1., -1.],
           [-1., -1., -1.,  ..., -1., -1., -1.],
           [-1., -1., -1.,  ..., -1., -1., -1.]],
  
          [[-1., -1., -1.,  ..., -1., -1., -1.],
           [-1., -1., -1.,  ..., -1., -1., -1.],
           [-1., -1., -1.,  ..., -1., -1., -1.],
           ...,
           [-1., -1., -1.,  ..., -1., -1., -1.],
           [-1., -1., -1.,  ..., -1., -1., -1.],
           [-1., -1., -1.,  ..., -1., -1., -1.]],
  
          [[-1., -1., -1.,  ..., -1., -1., -1.],
           [-1., -1., -1.,  ..., -1., -1., -1.],
           [-1., -1., -1.,  ..., -1., -1., -1.],
           ...,
           [-1., -1., -1.,  ..., -1., -1., -1.],
           [-1., -1., -1.,  ..., -1., -1., -1.],
           [-1., -1., -1.,  ..., -1., -1., -1.]]])],
 'image_1': [tensor([[[-0.7412, -0.7412, -0.741

In [36]:
n.index_mapper[50]

(29, 0)